In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from contextlib import redirect_stderr
from nltk import FreqDist
from nltk import word_tokenize
import pandas as pd
import numpy as np
import glob
import io
import re
df_us_10k = pd.read_csv('D:\\data\\finance\\financial_statements\\us\\SEC-preprocessed-download\\10k_2009_2019.csv', encoding='utf-8')
df_us_10k = df_us_10k[['adsh', 'cik', 'gvkey', 'form', 'filed', 'co_conm']].drop_duplicates()
df_kospi_10k_1115 = pd.read_csv('D:\\data\\finance\\financial_statements\\kr\\kospi200_2011_2015_translated.csv', encoding='utf-8')

In [2]:
def will_occurences(string):
    return len(re.findall('(?!A\s*|The\s*)(will)', string))

def file_to_string(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
          data = f.read()
    return data

path = 'D:\\data\\finance\\financial_statements\\us\\EDGAR-raw\\'
grammatic_tokens = ['will', 'shall', 'going_to', 'expect_to', 'expected_to', 'expecting_to']
gramm_tf = {token: [] for token in grammatic_tokens}
err = []
err_flag = False
for idx, row in df_us_10k.iterrows():
    try:
        string = file_to_string(path + row.adsh + '.txt')
    except:
        print('error on '+ str(row.adsh))
        err += [row.adsh]
        err_flag = True
    for token in gramm_tf.keys():
        if err_flag:
            gramm_tf[token] += [None]
        else:
            if token == 'will':
                gramm_tf[token] += [will_occurences(string)]
            else:
                gramm_tf[token] += [string.count(token)]
    err_flag = False
    if idx % 1000 == 0:
        print(idx)

for token in gramm_tf.keys():
    df_us_10k[token] = gramm_tf[token]

df_us_10k.to_csv('10k_grammatical_index.csv', encoding='utf-8', index=False)

0
1000
2000
error on 0001274494-10-000029
error on 0000768251-10-000008
error on 0001274494-10-000039
3000
4000
5000
6000
7000
8000
9000
error on 0000023217-13-000005
10000
error on 0001123360-13-000031
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
22000
23000
24000
25000
26000


Don't forget to normalise

# 10-k

In [ ]:
#tf-idf with skelarn - 
vectorizer = TfidfVectorizer(vocabulary={'will':0, 'shall':1})
X = vectorizer.fit_transform(df_i_want['text'])

In [24]:
df_us_tf = pd.DataFrame(columns=('adsh', 'gvkey', 'cik', 'filed'))
df_us_tf[['adsh', 'gvkey', 'cik', 'filed']] = df_us_10k[['adsh', 'gvkey', 'cik', 'filed']].dropna()
df_us_tf['will'] = df_us_10k['text'].dropna().apply(will_occurences)
df_us_tf['shall'] = df_us_10k['text'].dropna().str.count('shall')
df_us_tf['going_to'] = df_us_10k['text'].dropna().str.count('going to')

In [25]:
df_us_tf['expect_to'] = df_us_10k['text'].dropna().str.count('expect to')
df_us_tf['expected_to'] = df_us_10k['text'].dropna().str.count('expected to')
df_us_tf['expecting_to'] = df_us_10k['text'].dropna().str.count('expecting to')

In [26]:
df_us_tf[['will', 'shall', 'going_to', 'expect_to', 'expecting_to', 'expected_to']].mean()

will            18.653786
shall            0.150051
going_to         0.004285
expect_to        0.594213
expecting_to     0.002124
expected_to      3.090782
dtype: float64

In [13]:
df_us_tf.filed = pd.to_datetime(df_us_tf.filed, format='%Y%m%d')
df_us_tf[df_us_tf.will==0].groupby(df_us_tf.filed.dt.year).count()
#will이 1개도 등장하지 않는 애들이 약 30~40개

,adsh,cik,filed,will,shall,going_to
filed,,,,,,
2009,25,25,25,25,25,25
2010,51,51,51,51,51,51
2011,38,38,38,38,38,38
2012,47,47,47,47,47,47
2013,59,59,59,59,59,59
2014,36,36,36,36,36,36
2015,18,18,18,18,18,18
2016,27,27,27,27,27,27
2017,28,28,28,28,28,28


# KOSPI

In [63]:
df_kospi_10k_1115[df_kospi_10k_1115.Translated.isnull()]
df_kr_tf = pd.DataFrame(columns=['file'])
df_kr_tf[['file']] = df_kospi_10k_1115[['File']].dropna()
df_kr_tf['will'] = df_kospi_10k_1115['Translated'].dropna().apply(will_occurences)
df_kr_tf['shall'] = df_kospi_10k_1115['Translated'].dropna().str.count('shall')
df_kr_tf['going_to'] = df_kospi_10k_1115['Translated'].dropna().str.count('going to')

In [67]:
df_kr_tf[['will', 'shall', 'going_to']].mean()

will        7.084703
shall       0.119014
going_to    0.021444
dtype: float64

# Portfolio Performance

## 10-k

In [32]:
price_df = pd.read_csv('D:\\data\\finance\\price\\prices_08_19.csv')
stocks_of_interest = df_us_tf.gvkey.unique()
price_df['datadate'] = pd.to_datetime(price_df['datadate'], format='%Y%m%d')
price_df = price_df[price_df.gvkey.isin(stocks_of_interest)]
price_df['adj_closing_price'] = price_df['prccd'] / price_df['ajexdi']
price_df = price_df[['gvkey', 'datadate', 'tic', 'adj_closing_price']]
price_df

C:\Users\sylim2357\anaconda3\envs\future-tense-mining\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
df_us_gramm_index = df_us_tf[['gvkey', 'filed', 'will', 'shall', 'going_to', 'expect_to', 'expecting_to']].dropna()
df_us_gramm_index.filed = pd.to_datetime(df_us_gramm_index.filed, format='%Y%m%d')
df_us_gramm_index

,gvkey,filed,will,shall,going_to,expect_to,expecting_to
0,4843,2009-05-29,1.0,0.0,0.0,0.0,0.0
1,2574,2009-04-15,11.0,0.0,0.0,0.0,0.0
2,12540,2009-06-26,5.0,0.0,0.0,0.0,0.0
4,7881,2009-04-30,5.0,0.0,0.0,1.0,0.0
5,2991,2009-05-07,3.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
26415,34443,2019-10-25,54.0,0.0,0.0,0.0,0.0
26416,34765,2019-11-12,63.0,0.0,0.0,0.0,0.0
26417,35168,2019-10-31,61.0,0.0,0.0,0.0,0.0
26418,35035,2019-11-12,29.0,0.0,0.0,0.0,0.0


In [44]:
fidq_dates_suffix_list = ['-01-20 00:00:00','-04-20 00:00:00','-07-20 00:00:00','-10-20 00:00:00']
years = list(set([x.year for x in df_us_gramm_index['filed']]))
years.sort()
fidq_dates_list = [pd.Timestamp(str(years[0]-1))] + [pd.Timestamp(str(year)+suffix) for year in years for suffix in fidq_dates_suffix_list] + [pd.Timestamp(str(years[-1]+1)+fidq_dates_suffix_list[0])]
fidq_dates_list

[Timestamp('2008-01-01 00:00:00'),
 Timestamp('2009-01-20 00:00:00'),
 Timestamp('2009-04-20 00:00:00'),
 Timestamp('2009-07-20 00:00:00'),
 Timestamp('2009-10-20 00:00:00'),
 Timestamp('2010-01-20 00:00:00'),
 Timestamp('2010-04-20 00:00:00'),
 Timestamp('2010-07-20 00:00:00'),
 Timestamp('2010-10-20 00:00:00'),
 Timestamp('2011-01-20 00:00:00'),
 Timestamp('2011-04-20 00:00:00'),
 Timestamp('2011-07-20 00:00:00'),
 Timestamp('2011-10-20 00:00:00'),
 Timestamp('2012-01-20 00:00:00'),
 Timestamp('2012-04-20 00:00:00'),
 Timestamp('2012-07-20 00:00:00'),
 Timestamp('2012-10-20 00:00:00'),
 Timestamp('2013-01-20 00:00:00'),
 Timestamp('2013-04-20 00:00:00'),
 Timestamp('2013-07-20 00:00:00'),
 Timestamp('2013-10-20 00:00:00'),
 Timestamp('2014-01-20 00:00:00'),
 Timestamp('2014-04-20 00:00:00'),
 Timestamp('2014-07-20 00:00:00'),
 Timestamp('2014-10-20 00:00:00'),
 Timestamp('2015-01-20 00:00:00'),
 Timestamp('2015-04-20 00:00:00'),
 Timestamp('2015-07-20 00:00:00'),
 Timestamp('2015-10-

In [ ]:
submission_df = pd.read_json('C:\\Users\\sylim2357\\Documents\\SNU\\Projects\\FVID\\data\\KRX\\kospi200_configs\\kospi_submission_date.json')
df_sentiment = pd.read_csv('C:\\Users\\sylim2357\\Documents\\SNU\\Projects\\FVID\\data\\KRX\\kospi_sentiment\\kospi_sentiment_2011_2019.csv')
market_days = pd.to_datetime(pd.read_csv('C:\\Users\\sylim2357\\Documents\\SNU\\Projects\\FVID\\data\\KRX\\date_normal.csv')['Date'])

submission_df = submission_df[~submission_df['file_name'].str.contains(']')]
submission_df['file_name'] = submission_df['file_name'].str.split('.pdf', expand=True).iloc[:,0]
submission_df['company_code'] = submission_df['file_name'].str.split('_',expand=True).iloc[:,0]
submission_df['submission_date'] = pd.to_datetime(submission_df['submission_date'], format='%Y%m%d')
#constructing rebalancing dates
fidq_dates_suffix_list = ['-01-20 00:00:00','-04-20 00:00:00','-07-20 00:00:00','-10-20 00:00:00']
years = list(set([x.year for x in submission_df['submission_date']]))
years.sort()
fidq_dates_list = [pd.Timestamp(str(years[0]-1))] + [pd.Timestamp(str(year)+suffix) for year in years for suffix in fidq_dates_suffix_list] + [pd.Timestamp(str(years[-1]+1)+fidq_dates_suffix_list[0])]
submission_df['rebalance_date'] = pd.cut(submission_df['submission_date'], fidq_dates_list, labels = fidq_dates_list[1:])
#quantiling the sentiments per rebalance date
sentiment_submission_df = submission_df.merge(df_sentiment, left_on='file_name', right_on='File').drop('File', axis=1)
sentiment_submission_df['polarity_quantile'] = sentiment_submission_df.groupby('rebalance_date', level=0).apply(pd.DataFrame.sort_values, 'Polarity', ascending=False)['Polarity'].transform(lambda x: pd.qcut(x, 4, labels=range(1,5)))
#filtering only top and bottom quantiles
sentiment_submission_top_bottom_df = sentiment_submission_df[sentiment_submission_df.polarity_quantile.isin([1,4])]
sentiment_submission_top_bottom_df['rebalance_date'] = pd.to_datetime(sentiment_submission_top_bottom_df['rebalance_date'])